In [10]:
import copy

import numpy as np

def parse_board(board):
    return np.array([[int(c) if c.isdigit() else 0 for c in row.strip() if c != " "] for row in board.strip().split('\n')])

def print_board(board):
    for i, row in enumerate(board):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - -")
        for j, val in enumerate(row):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            if j == 8:
                print(val)
            else:
                print(str(val) + " ", end="")
    print()

def board_is_solved(board):

    for num in range(1, 10):
        # Check rows
        for i in range(9):
            count = 0
            for j in range(9): #cycle through columns of a fixed row
                if board[i][j] == num:
                    count += 1
                if count > 1: #if a number appear more than once in a row
                    return False
            if count < 1: #if a number doesn't appear in a row
                return False
                    
        # check columns
        for j in range(9):
            count = 0
            for i in range(9): #cycle through rows of a fixed column
                if board[i][j] == num:
                    count += 1
                if count > 1: #if a number appear more than once in a row
                    return False
            if count < 1: #if a number doesn't appear in a row
                return False

    # Check boxes
        for box_x in range(1, 3):
            for box_y in range(1, 3):
                count = 0
                for i in range(box_y * 3, box_y * 3 + 3):
                    for j in range(box_x * 3, box_x * 3 + 3):
                        if board[i][j] == num:
                            count += 1
                        if(count > 1): #if a number appears more than one time in a box 3x3
                            return False
                if count < 1: #if a number doesn't appear in a box
                    return False
    return True


def board_is_full(board):
    
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return False
    return True

In [11]:
import random

# Genetic Algorithm parameters
POPULATION_SIZE = 50
MUTATION_RATE = 0.3
MAX_GENERATIONS = 1000

def coords_already_full(board):
    
    list = []
    
    for i in range(9):
        for j in range(9):
            if board[i][j] != 0:
                list.append((i,j))
    
    return list
    
def fitness(board, already_filled):
    fitness_score = 0
    
    for i in range(9):
        for j in range(9):
            if not (i,j) in already_filled:
                
                # check collision in row
                for j2 in range(9):
                    # if the element found (different from the starting element) has equal value it's a collision
                    if board[i, j2] == board[i][j] and (i,j2) != (i,j): 
                        fitness_score -= 1
                        
                # check collision in column
                for i2 in range(9):
                    if board[i2, j] == board[i][j] and (i2,j) != (i,j):
                        fitness_score -= 1
                        
                #check collision in box 3x3
                box_x = j // 3
                box_y = i // 3
                for i2 in range(box_y*3, box_y*3 + 3):
                    for j2 in range(box_x*3, box_x*3 + 3):
                        if board[i2, j2] == board[i][j] and (i2,j) != (i,j):
                            fitness_score -= 1
    
    return fitness_score

def mutate(board, already_filled):
    
    while True:
        (x,y) = (random.randint(0, 8), random.randint(0, 8))
        if not (x,y) in already_filled:
            break
    
    new_board = copy.deepcopy(board)
    new_board[x][y] = random.randint(1,9)
    
    return new_board
    
# todo chiedere a Claude se crossover è meglo farlo verticale, orizzontale o a box, o se selezionare verticale/orizzontale a random
# Vertical crossover of the board
def crossover(parent1, parent2, already_filled):
    
    child = copy.deepcopy(parent1)
    #crossover_point = random.randint(1, 8)
    #for i in range(9):
    #    for j in range(crossover_point, 8):
    #        child[i][j] = parent2[i][j]
    for i in range(9):
        for j in range(9):
            if not (i,j) in already_filled:
                if random.random() < MUTATION_RATE:
                    child[i][j] = parent2[i][j]
                
    return child

# Select parents for crossover using tournament selection
def tournament_selection(population):
    tournament_size = 20
    tournament = random.sample(population, tournament_size)
    return max(tournament, key=lambda x: x[1])

# generate a new board with random numbers (where numbers were not already set)
def generate_board_state(initial_board):
    
    new_board = copy.deepcopy(initial_board)
    
    for i in range(9):
        for j in range(9):
            if new_board[i][j] == 0:
                new_board[i][j] = random.randint(1, 9)
    
    return new_board


### TEST SECTION

In [12]:
# Example Board
sudoku_board = """
37. 5....6
... 36. .12
... .91 75.
... 154 .7.
..3 .7. 6..
.5. 638 ...
.64 98. ...
59. .26 ...
2.. ..5 .64
"""

sudoku = """
8........
..36.....
.7..9.2..
.5...7...
....45.7.
...1...3.
..1....68
..85...1.
.9....4..
"""

board = parse_board(sudoku_board)

# Generate the initial population
population = [(generate_board_state(board), 0) for _ in range(POPULATION_SIZE)]

# save in a var the coordinates to the cells already filled
already_filled = coords_already_full(board)
# set the best board to the initial board
best_board_state = board

# Main Genetic Algorithm loop
for generation in range(MAX_GENERATIONS):
    # Calculate fitness for each board state
    population = [(board_state, fitness(board_state, already_filled)) for board_state, _ in population]

    # Check if solution is found
    best_board_state = max(population, key=lambda x: x[1])[0]
    if fitness(best_board_state, already_filled) == 0:
        print("Solution found in generation", generation)
        break
        
    print("Generazione ", generation)
    print("Fitness: ", max(population, key=lambda x: x[1])[1])
    print_board(best_board_state)
    print()
        
    # Create the next generation
    new_population = []

    # Elitism: Keep the best board state from the previous generation
    new_population.append(max(population, key=lambda x: x[1]))
    
    # Perform selection, crossover, and mutation
    while len(new_population) < POPULATION_SIZE:
        parent1 = tournament_selection(population)
        parent2 = tournament_selection(population)
        child = crossover(parent1[0], parent2[0], already_filled)
        if random.random() < MUTATION_RATE:
            child = mutate(child, already_filled)
        new_population.append((child, 0))

    # Update the population
    population = new_population

# Print the best solution
print("Best solution:", best_board_state)

Generazione  0
Fitness:  -92
3 7 9 | 5 1 8 | 4 7 6
3 6 4 | 3 6 7 | 5 1 2
8 1 3 | 2 9 1 | 7 5 7
- - - - - - - - - - - -
3 5 3 | 1 5 4 | 7 7 2
9 8 3 | 7 7 6 | 6 4 3
7 5 6 | 6 3 8 | 4 7 1
- - - - - - - - - - - -
8 6 4 | 9 8 5 | 4 7 3
5 9 4 | 4 2 6 | 9 3 7
2 9 7 | 5 3 5 | 8 6 4


Generazione  1
Fitness:  -79
3 7 3 | 5 8 9 | 5 6 6
7 9 2 | 3 6 4 | 1 1 2
1 9 5 | 7 9 1 | 7 5 8
- - - - - - - - - - - -
9 5 2 | 1 5 4 | 8 7 4
4 3 3 | 8 7 2 | 6 6 4
1 5 6 | 6 3 8 | 2 2 5
- - - - - - - - - - - -
7 6 4 | 9 8 7 | 2 5 3
5 9 6 | 3 2 6 | 6 4 9
2 2 8 | 3 2 5 | 7 6 4


Generazione  2
Fitness:  -73
3 7 3 | 5 4 9 | 5 6 6
7 9 7 | 3 6 4 | 1 1 2
1 2 5 | 9 9 1 | 7 5 4
- - - - - - - - - - - -
9 6 2 | 1 5 4 | 8 7 1
4 3 3 | 8 7 2 | 6 6 5
5 5 6 | 6 3 8 | 3 2 9
- - - - - - - - - - - -
7 6 4 | 9 8 7 | 5 5 3
5 9 7 | 6 2 6 | 3 8 9
2 2 8 | 8 2 5 | 7 6 4


Generazione  3
Fitness:  -66
3 7 3 | 5 4 9 | 5 6 6
7 9 7 | 3 6 4 | 1 1 2
1 2 5 | 7 9 1 | 7 5 4
- - - - - - - - - - - -
9 5 2 | 1 5 4 | 8 7 1
4 3 3 | 8 7 2 | 6 6 5
1 5 6 

In [13]:
#print("Example Sudoku:")
#print_board(board)
#
#
#if(solve_board(board)):
#    print("Soluzione: ")
#    print_board(board)
#else:
#    print("Board non risolvibile")